# Day 2: RAG

### What is RAG? 

**RAG or Retrieval Augmented Generation** - is a technique that is widely used in industry to help reduce model hallucination.
For example, have you ever tryed to ask ChatGPT as questions about something that you know about and the answer is just made up?
This is what we call a **hallucination** and RAG is a technique that we use to reduce hallucination. 

RAG does this by finding relavant chuncks of text to add to your prompt before sending it off to a LLM.

**Why is this useful?** 
Models like ChatGPT are training on the internet. What if you ask it a questions, which it wasn't training on? Can you think of such questions? 
For example, let's say you have a business with your own data. You don't share any of that data on the internet or to anyone outside your business. 
How can you create a chatbot that let's you have a converation with your business data? You use RAG!

Today we will learn about how you process document, put them into a database and how you retrieve those documents from a database. Let's get started! 🚀

In [1]:
# utilities
from rich import inspect

# import core langchain libraries
from langchain_community.document_loaders import PyPDFLoader                    # special method to load PDF files
from langchain_community.document_loaders import UnstructuredHTMLLoader         # special method to load HTML
from langchain_community.document_loaders import UnstructuredMarkdownLoader     # special method to load Markdown

from langchain_community.document_loaders import DirectoryLoader                # helps use load multiple documents from a directory
from langchain_community.document_loaders.merge import MergedDataLoader         # helps merge multiple different types of documents

---
# Section 1: Loading Documents

## 📃 Step 1: Simple loading documents

### PDF document loading

+ [Loading PDF files reference](https://python.langchain.com/docs/how_to/document_loader_pdf/)

In [2]:
# Step 1: Set the file path for the documents you want to load
file_path_pdf = "../data/principles-for-navigating-big-debt-crises-by-ray-dalio.pdf"

# Step 2: Create loader that is specific for your document type
loader_pdf = PyPDFLoader(file_path_pdf)

# Step 3: Load in the document
my_pdf_document = loader_pdf.load()

In [3]:
# Step 4: Examine what we just did
#         'my_document' - it contains metadata, source, page number and the content of the page we just "scrapped"

my_pdf_document[0]

Document(metadata={'source': '../data/principles-for-navigating-big-debt-crises-by-ray-dalio.pdf', 'page': 0}, page_content='Principles For Navigating\nBIG DEBT CRISES\nPart 1:\nThe Archetypal Big Debt Cycle')

In [4]:
print("Source:       ", my_pdf_document[0].metadata["source"])
print("Page number:  ",   my_pdf_document[0].metadata["page"])
print("Page content: ", my_pdf_document[0].page_content)

Source:        ../data/principles-for-navigating-big-debt-crises-by-ray-dalio.pdf
Page number:   0
Page content:  Principles For Navigating
BIG DEBT CRISES
Part 1:
The Archetypal Big Debt Cycle


### HTML document loading

In [5]:
# Step 1: Setting the file path
file_path_html = "../data/Is_It_Wrong_to_Remove_a_Card_From_Monopoly_The_New_York_Times.html"

# Step 2: Create loader that is specific for your document type
loader_html = UnstructuredHTMLLoader(file_path_html)

# Step 3: Load in the document
my_html_document = loader_html.load()

In [6]:
print("Source:       ", my_html_document[0].metadata["source"])
print("Page content: ", my_html_document[0].page_content[:100])

Source:        ../data/Is_It_Wrong_to_Remove_a_Card_From_Monopoly_The_New_York_Times.html
Page content:  Magazine|Is It Wrong to Remove a Card From Monopoly?

https://www.nytimes.com/2025/04/16/magazine/mo


### Markdown document loading

In [7]:
# Step 1: Setting the file path
file_path_markdown = "../data/README.md"

# Step 2: Create loader that is specific for your document type
loader_markdown = UnstructuredMarkdownLoader(file_path_markdown)

# Step 3: Load in the document
my_markdown_document = loader_markdown.load()

In [8]:
my_markdown_document

[Document(metadata={'source': '../data/README.md'}, page_content='Release Notes\n\nCI\n\nPyPI - License\n\nPyPI - Downloads\n\nGitHub star chart\n\nOpen Issues\n\nOpen in Dev Containers\n\n\n\nTwitter\n\nCodSpeed Badge\n\n[!NOTE] Looking for the JS/TS library? Check out LangChain.js.\n\nLangChain is a framework for building LLM-powered applications. It helps you chain together interoperable components and third-party integrations to simplify AI application development — all while future-proofing decisions as the underlying technology evolves.\n\nbash pip install -U langchain\n\nTo learn more about LangChain, check out the docs. If you’re looking for more advanced customization or agent orchestration, check out LangGraph, our framework for building controllable agent workflows.\n\nWhy use LangChain?\n\nLangChain helps developers build applications powered by LLMs through a standard interface for models, embeddings, vector stores, and more.\n\nUse LangChain for: - Real-time data augmenta

## 🦜 Langchain provide ways of loading in all types of documents 

Here is a reference for loading documents that I didn't cover in the examples. When we begin to work on our group project these links can be useful to help you load all types of document types.

#### Reference: [Document loaders](https://python.langchain.com/docs/how_to/#document-loaders)
+ [How to: load PDF files](https://python.langchain.com/docs/how_to/document_loader_pdf/)
+ [How to: load web pages](https://python.langchain.com/docs/how_to/document_loader_web/)
+ [How to: load CSV data](https://python.langchain.com/docs/how_to/document_loader_csv/)
+ [How to: load HTML data](https://python.langchain.com/docs/how_to/document_loader_html/)
+ [How to: load JSON data](https://python.langchain.com/docs/how_to/document_loader_json/)
+ [How to: load Markdown data](https://python.langchain.com/docs/how_to/document_loader_markdown/)

## 📂 Step 2: Loading documents from a folder

**Scenario:** What if you have a folder full of documents and you want to load in all of the PDF documents you have at the same time?

<div style="text-align:center;">
    <img src="../assets/image1.png" width="25%" height="25%"/>
</div>

### 2.1: Let's say we want to load in all of the PDF files we have in a folder into a document loader.

In [9]:
# Step 1: Using Langchain's DirectoryLoader set the path to your folder and use
pdfs_file_path = "../data/"

# Step 2: use Directory loader to load in all of the documents from a folder

loader_pdfs = DirectoryLoader(pdfs_file_path,                # set your base file path
                              glob="*.pdf",                  # *.pdf means all files with the extension .pdf 
                              show_progress=True,            # progress bar
                              use_multithreading=True,       # load from disk using multiple threads
                              loader_cls=PyPDFLoader         # add the langchain PDF class
                             )

# Step 3: Load in the documents
my_pdf_documents = loader_pdfs.load()

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:12<00:00,  4.10s/it]


#### Let's talk about what just happend.

In this project - there is a folder called `data/`. That folder contains three different documents that end in a .pdf.
    
+ data/principles-for-navigating-big-debt-crises-by-ray-dalio.pdf
+ data/react.pdf
+ data/trends_in_deep_learning_nlp.pdf

`DirectoryLoader` took all of the documents ending in .pdf, saved them to a list of `Document` objects.

In [10]:
my_pdf_documents[0]

Document(metadata={'source': '../data/react.pdf', 'page': 0}, page_content='Published as a conference paper at ICLR 2023\nREAC T: S YNERGIZING REASONING AND ACTING IN\nLANGUAGE MODELS\nShunyu Yao∗*,1, Jeffrey Zhao2, Dian Yu2, Nan Du2, Izhak Shafran2, Karthik Narasimhan1, Yuan Cao2\n1Department of Computer Science, Princeton University\n2Google Research, Brain team\n1{shunyuy,karthikn}@princeton.edu\n2{jeffreyzhao,dianyu,dunan,izhak,yuancao}@google.com\nABSTRACT\nWhile large language models (LLMs) have demonstrated impressive performance\nacross tasks in language understanding and interactive decision making, their\nabilities for reasoning (e.g. chain-of-thought prompting) and acting (e.g. action\nplan generation) have primarily been studied as separate topics. In this paper, we\nexplore the use of LLMs to generate both reasoning traces and task-speciﬁc actions\nin an interleaved manner, allowing for greater synergy between the two: reasoning\ntraces help the model induce, track, and up

In [11]:
# We can also look at the details of the Document object
print("Source of the text:  " , my_pdf_documents[0].metadata["source"])
print("Page:                " , my_pdf_documents[0].metadata["page"])

Source of the text:   ../data/react.pdf
Page:                 0


#### What if you have a scenario where you want to add more data to the `Document` object? 

+ For example - your data is missing the authors name, but you know the author of the document and want to add it to use if for a later task - for example, as a reference.

In [12]:
my_pdf_documents[0].metadata["author"] = "Andronik"

In [13]:
my_pdf_documents[0].metadata["author"]

'Andronik'

In [14]:
my_pdf_documents[0]

Document(metadata={'source': '../data/react.pdf', 'page': 0, 'author': 'Andronik'}, page_content='Published as a conference paper at ICLR 2023\nREAC T: S YNERGIZING REASONING AND ACTING IN\nLANGUAGE MODELS\nShunyu Yao∗*,1, Jeffrey Zhao2, Dian Yu2, Nan Du2, Izhak Shafran2, Karthik Narasimhan1, Yuan Cao2\n1Department of Computer Science, Princeton University\n2Google Research, Brain team\n1{shunyuy,karthikn}@princeton.edu\n2{jeffreyzhao,dianyu,dunan,izhak,yuancao}@google.com\nABSTRACT\nWhile large language models (LLMs) have demonstrated impressive performance\nacross tasks in language understanding and interactive decision making, their\nabilities for reasoning (e.g. chain-of-thought prompting) and acting (e.g. action\nplan generation) have primarily been studied as separate topics. In this paper, we\nexplore the use of LLMs to generate both reasoning traces and task-speciﬁc actions\nin an interleaved manner, allowing for greater synergy between the two: reasoning\ntraces help the model

### 2.2: Let's do something similar but with HTML files.

In [15]:
# Step 1: Using Langchain's DirectoryLoader set the path to your folder and use
html_file_path = "../data/"

# Step 2: use Directory loader to load in all of the documents from a folder

loader_html = DirectoryLoader(html_file_path,                       
                              glob="*.html",
                              show_progress=True,
                              use_multithreading=True,
                              loader_cls=UnstructuredHTMLLoader              # the only thing that changes from the top example
                             )

# Step 3: Load in the documents
my_html_documents = loader_html.load()

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.81it/s]


In [16]:
# Same applies as above example
print("Source of the text:  " , my_html_documents[0].metadata["source"])

Source of the text:   ../data/Is_It_Wrong_to_Remove_a_Card_From_Monopoly_The_New_York_Times.html


In [17]:
my_html_documents[0].page_content[:1000]

'Magazine|Is It Wrong to Remove a Card From Monopoly?\n\nhttps://www.nytimes.com/2025/04/16/magazine/monopoly-games-children-ethics.html\n\nAdvertisement\n\nSKIP ADVERTISEMENT\n\nSubscriber-only Newsletter\n\nThe Ethicist\n\nIs It Wrong to Remove a Card From Monopoly?\n\nThe magazine’s Ethicist columnist on altering board games to teach children ethical behavior.\n\nKwame Anthony Appiah\n\nBy Kwame Anthony Appiah\n\nApril 16, 2025\n\nYou’re reading The Ethicist newsletter, for Times subscribers only. Advice on life’s trickiest situations and moral dilemmas from the philosopher Kwame Anthony Appiah.\n\nMy grandchildren love playing Monopoly. The board game has become a great way for me to interact with them, and also a great way for them to see capitalism in all its imperfect glory. The problem: One of the cards a player may draw when landing on Community Chest is “Bank Error in Your Favor. Collect $200.” Right when we first started playing the game together, I removed that card from th

## 🗂️ Step 3: Loading different documents from the same folder 

In [18]:
# Step 1: Loading in the PDF, HTML and Markdown files
file_path_data = "../data/"

loader_pdfs = DirectoryLoader(file_path_data,              
                              glob="*.pdf",              
                              show_progress=True,            
                              use_multithreading=True,     
                              loader_cls=PyPDFLoader)                   # on difference is the loader class


loader_html = DirectoryLoader(file_path_data,                       
                              glob="*.html",
                              show_progress=True,
                              use_multithreading=True,
                              loader_cls=UnstructuredHTMLLoader)


loader_markdown = DirectoryLoader(file_path_data,                       
                                  glob="*.md",
                                  show_progress=True,
                                  use_multithreading=True,
                                  loader_cls=UnstructuredMarkdownLoader)



# Step 2: User MergeDataLoader to combine all of the different loader types you have for your data
my_loaders = MergedDataLoader(loaders=[loader_pdfs, loader_html, loader_markdown])

# Step 3: Load all documents into a list of Document objects
my_docs = my_loaders.load()

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 78.04it/s]


In [19]:
print("The length of my list of Documents: ", len(my_docs))

The length of my list of Documents:  548


In [20]:
i = 0
while i < 10:
    
    for doc in my_docs:
        print(doc.metadata)
        break
    
    i += 1

{'source': '../data/react.pdf', 'page': 0}
{'source': '../data/react.pdf', 'page': 0}
{'source': '../data/react.pdf', 'page': 0}
{'source': '../data/react.pdf', 'page': 0}
{'source': '../data/react.pdf', 'page': 0}
{'source': '../data/react.pdf', 'page': 0}
{'source': '../data/react.pdf', 'page': 0}
{'source': '../data/react.pdf', 'page': 0}
{'source': '../data/react.pdf', 'page': 0}
{'source': '../data/react.pdf', 'page': 0}


---
# Section 2: RAG

What is RAG and why is it important? Let's say that you are working at a company and your boss want you to create a chatbot that is able to answer questions about your business. For example, how much inventory do we have left in the parts department? What are the type of defects we had for Part A in the month of March? 

To answer your bosses questions using a chatbot your chatbot needs to be up to date on the information your business has. Plus, the data your business has is not on the internet!

RAG is a way of adding extra knowledge to the LLM without needing to retrain the entire model, which is expensive, time consuming and requires expertise. What RAG provides use is a database of additional information which we can hookup to the LLM and use to answer questions.

Here is a diagram and next we will talk about how it works.

![rag](../assets/rag.png)


Three important parts to a RAG:
1. **Document Splitting** - how we chop of the documents into smaller pieces.
2. **Embeddings** - how we take those smaller pieces and put them into a database.
3. **Retrieval** - how we retrieve pieces of our documents from the RAG


If you want more details RAG. Here is a paper you can check out...
+ [What is a RAG?](https://www.youtube.com/watch?v=T-D1OfcDW1M)
+ [Searching for Best Practices in Retrieval-Augmented
Generation](https://arxiv.org/pdf/2407.01219)

## Step 1: Chunking Documents and Embeddings

Once we know the documents that we want to include in our database, we need to processes the documents.
**Chunking** is a way of splitting your documents into smaller pieces so when you need to retrieve something from your database you don't collect the entire document. You just collect a piece of it.

<div style="text-align:center;">
    <img src="../assets/text_splitting.png" width="" height=""/>
</div>

### Step 1.1: Recursive Chunking

In [21]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [22]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=10,
    chunk_overlap=5
)

In [23]:
test_sentence = "Hello. My name is Andronik and I have a cat named Jinxy"

In [24]:
texts = text_splitter.split_text(test_sentence)

In [25]:
for n in [4, 8, 16]:
    
    text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=n,
    chunk_overlap=n//2)

    text = text_splitter.split_text(test_sentence)

    for t in text:
        print(f"Chunk length: {len(t)}, Text Split: {t}")

    print("------------------------------------------------------------")

Chunk length: 4, Text Split: Hell
Chunk length: 4, Text Split: llo.
Chunk length: 2, Text Split: My
Chunk length: 3, Text Split: nam
Chunk length: 3, Text Split: ame
Chunk length: 2, Text Split: is
Chunk length: 3, Text Split: And
Chunk length: 4, Text Split: ndro
Chunk length: 4, Text Split: roni
Chunk length: 3, Text Split: nik
Chunk length: 3, Text Split: and
Chunk length: 1, Text Split: I
Chunk length: 3, Text Split: hav
Chunk length: 3, Text Split: ave
Chunk length: 1, Text Split: a
Chunk length: 3, Text Split: cat
Chunk length: 3, Text Split: nam
Chunk length: 4, Text Split: amed
Chunk length: 3, Text Split: Jin
Chunk length: 4, Text Split: inxy
------------------------------------------------------------
Chunk length: 6, Text Split: Hello.
Chunk length: 7, Text Split: My name
Chunk length: 2, Text Split: is
Chunk length: 7, Text Split: Androni
Chunk length: 5, Text Split: ronik
Chunk length: 5, Text Split: and I
Chunk length: 6, Text Split: I have
Chunk length: 5, Text Split: a 

### Step 1.2: Semantic Chunking

In [26]:
# loading in the api keys to connect to OpenAI
import os
from dotenv import load_dotenv
load_dotenv()

True

In [27]:
from langchain_openai import OpenAIEmbeddings
from langchain_experimental.text_splitter import SemanticChunker

In [28]:
# connect to OpenAI 
openai_api_key = os.environ.get("OPENAI_API_KEY")
openai_organization = os.environ.get("OPENAI_ORGANIZATION")

In [29]:
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key,
                          model="text-embeddings-3-small")

In [30]:
text_splitter = SemanticChunker(OpenAIEmbeddings())

In [31]:
test_sentence = """
This raises the question: Can function inlining affect the output of a numerical program? 
I’m interested in both the programmer inlining a function manually in his editor and the compiler doing it for you during an optimization pass.
It turns out that yes, there are multiple ways in which inlining can change results, and specifics depend on the interplay of your language spec, 
compiler and hardware. One of the big reasons why compilers do inlining is that it increases the scope for optimizations. Most optimization passes 
run on individual functions, so inlining gives the compiler more code to work with, and more opportunities to apply potentially result-changing optimizations.
Here’s a concrete example that compiled with gcc -O3 -march=haswell produces different results depending on whether the function is inlined or not
"""

In [32]:
docs = text_splitter.split_text(test_sentence)

In [33]:
print("Number of chunks:", len(docs))
for doc in docs:
    print("--------------------------------------------------------------------")
    print(doc)
    print("--------------------------------------------------------------------")

Number of chunks: 2
--------------------------------------------------------------------

This raises the question: Can function inlining affect the output of a numerical program? I’m interested in both the programmer inlining a function manually in his editor and the compiler doing it for you during an optimization pass. It turns out that yes, there are multiple ways in which inlining can change results, and specifics depend on the interplay of your language spec, 
compiler and hardware. One of the big reasons why compilers do inlining is that it increases the scope for optimizations.
--------------------------------------------------------------------
--------------------------------------------------------------------
Most optimization passes 
run on individual functions, so inlining gives the compiler more code to work with, and more opportunities to apply potentially result-changing optimizations. Here’s a concrete example that compiled with gcc -O3 -march=haswell produces differe

### What did the `OpenAIEmbeddings()` do?

The basic idea is this...

The embeddings model take a look at the text that we give it and decides to devide it into 2 pieces. Each piece we can think of as an "idea". `Chunk 1` represents idea 2 and `Chunk 2` represents idea 2.

```mermaid
flowchart LR
node1[Document]
node2["Chunk 1"]
node3["Chunk 2"]
node1-->node2
node1-->node3
```

---

## Step 2: Creating a Database

![vdb](../assets/vdb.png)

In [34]:
from rich import inspect
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

In [35]:
# first initilize an embeddings model to use on your documents
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key,
                              model="text-embedding-3-large",
                              chunk_size=1000)

In [36]:
# Step 1: First initilize the SemanticChunker
text_splitter = SemanticChunker(embeddings,
                                breakpoint_threshold_type="percentile",
                                breakpoint_threshold_amount=70.0)

In [37]:
# Step 2: Split the documents into section. 
#   We are hoping each section contains a seperate idea.
docs = text_splitter.split_documents(my_docs)

In [38]:
for doc in docs:
    print(doc)
    break

page_content='Published as a conference paper at ICLR 2023
REAC T: S YNERGIZING REASONING AND ACTING IN
LANGUAGE MODELS
Shunyu Yao∗*,1, Jeffrey Zhao2, Dian Yu2, Nan Du2, Izhak Shafran2, Karthik Narasimhan1, Yuan Cao2
1Department of Computer Science, Princeton University
2Google Research, Brain team
1{shunyuy,karthikn}@princeton.edu
2{jeffreyzhao,dianyu,dunan,izhak,yuancao}@google.com
ABSTRACT
While large language models (LLMs) have demonstrated impressive performance
across tasks in language understanding and interactive decision making, their
abilities for reasoning (e.g. chain-of-thought prompting) and acting (e.g.' metadata={'source': '../data/react.pdf', 'page': 0}


In [39]:
print("New Length after chunking:", len(docs))

New Length after chunking: 3883


In [40]:
# Step 3: Insert your documents into a database after turning your docs into numbers.
vectorstore = Chroma.from_documents(
    documents=docs,
    collection_name="tumo_2025",
    embedding=embeddings,
    persist_directory="../db/my_first_vdb",  # Where to save data locally, remove if not necessary
)

In [41]:
# Let's run a test and see what the most similar documents to the phrase "Machine Learning"
vectorstore.similarity_search("Machine Learning")

[Document(id='84422d60-6162-431a-a264-b4125692440c', metadata={'page': 24, 'source': '../data/trends_in_deep_learning_nlp.pdf'}, page_content='25\nTABLE X: Comparison of ELMo + Baseline with the previous state of the art (SOTA) on various NLP tasks. The table has\nbeen adapted from [41]. SOTA results have been taken from [41]; SQUAD [166]: QA task; SNLI [178]: Stanford Natural\nLanguage Inference task; SRL [153]: Semantic Role Labelling; Coref [179]: Coreference Resolution; NER [180]: Named Entity\nRecognition; SST-5 [4]: Stanford Sentiment Treebank 5-class classiﬁcation;\nTask Previous SOTA Previous\nSOTA Results Baseline ELMo +\nBaseline\nIncrease\n(Absolute/Relative)\nSQuAD Liu et al. [181] 84.4 81.1 85.8 4.7 / 24.9%\nSNLI Qian et al. [182] 88.6 88.0 88.70 ±0.17 0.7 / 5.8%\nSRL Luheng et al. [183] 81.7 81.4 84.6 3.2 / 17.2%\nCoref Kenton et al. [184] 67.2 67.2 70.4 3.2 / 9.8%\nNER Matthew et al. [185] 91.93 ±0.19 90.15 92.22 ±0.10 2.06 / 21%\nSST-5 Bryan et al. [186] 53.7 51.4 54.7 

In [42]:
# Step 4: Langhchain as a retriever method that makes it easier to use the vectorbase 
#         by allowing it to be easier to integrate into our chain of functions. 

retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [43]:
my_retrieved_docs = retriever.invoke("Machine Learning")

In [44]:
print("How many documents we retrieved: ", len(my_retrieved_docs))

How many documents we retrieved:  5


In [45]:
for doc in my_retrieved_docs:
    print(doc)
    print("----------")

page_content='25
TABLE X: Comparison of ELMo + Baseline with the previous state of the art (SOTA) on various NLP tasks. The table has
been adapted from [41]. SOTA results have been taken from [41]; SQUAD [166]: QA task; SNLI [178]: Stanford Natural
Language Inference task; SRL [153]: Semantic Role Labelling; Coref [179]: Coreference Resolution; NER [180]: Named Entity
Recognition; SST-5 [4]: Stanford Sentiment Treebank 5-class classiﬁcation;
Task Previous SOTA Previous
SOTA Results Baseline ELMo +
Baseline
Increase
(Absolute/Relative)
SQuAD Liu et al. [181] 84.4 81.1 85.8 4.7 / 24.9%
SNLI Qian et al. [182] 88.6 88.0 88.70 ±0.17 0.7 / 5.8%
SRL Luheng et al. [183] 81.7 81.4 84.6 3.2 / 17.2%
Coref Kenton et al. [184] 67.2 67.2 70.4 3.2 / 9.8%
NER Matthew et al. [185] 91.93 ±0.19 90.15 92.22 ±0.10 2.06 / 21%
SST-5 Bryan et al. [186] 53.7 51.4 54.7 0.5 3.3 / 6.8%
Task BiLSTM+
ELMo+Attn BERT
QNLI 79.9 91.1
SST-2 90.9 94.9
STS-B 73.3 86.5
RTE 56.8 70.1
SQuAD 85.8 91.1
NER 92.2 92.8
TABLE XI: 

#### How to load in your database after you have created it

In [46]:
loaded_vectorstore = Chroma(
    persist_directory="../db/my_first_vdb",
    embedding_function=embeddings,
    collection_name="tumo_2025",
)

In [47]:
loaded_vectorstore.similarity_search_with_score("Machine Learning")

[(Document(id='84422d60-6162-431a-a264-b4125692440c', metadata={'page': 24, 'source': '../data/trends_in_deep_learning_nlp.pdf'}, page_content='25\nTABLE X: Comparison of ELMo + Baseline with the previous state of the art (SOTA) on various NLP tasks. The table has\nbeen adapted from [41]. SOTA results have been taken from [41]; SQUAD [166]: QA task; SNLI [178]: Stanford Natural\nLanguage Inference task; SRL [153]: Semantic Role Labelling; Coref [179]: Coreference Resolution; NER [180]: Named Entity\nRecognition; SST-5 [4]: Stanford Sentiment Treebank 5-class classiﬁcation;\nTask Previous SOTA Previous\nSOTA Results Baseline ELMo +\nBaseline\nIncrease\n(Absolute/Relative)\nSQuAD Liu et al. [181] 84.4 81.1 85.8 4.7 / 24.9%\nSNLI Qian et al. [182] 88.6 88.0 88.70 ±0.17 0.7 / 5.8%\nSRL Luheng et al. [183] 81.7 81.4 84.6 3.2 / 17.2%\nCoref Kenton et al. [184] 67.2 67.2 70.4 3.2 / 9.8%\nNER Matthew et al. [185] 91.93 ±0.19 90.15 92.22 ±0.10 2.06 / 21%\nSST-5 Bryan et al. [186] 53.7 51.4 54.7